In [1]:
# 주어진 데이터에 맞게 code1과 code2 그리고 유사성에 대한 pair을 읽어
# code에 대한 토큰화 작업과 토큰에 대한 index 작업
# 토큰에 대한 index 작업은 전체 code의 토큰에 대한 사전을 만든후 단어에 대한 index를 사용

import pandas as pd
import torch

# # pandas를 사용 open 폴더에 있는 sample_train.csv 파일 로드
train = pd.read_csv("open/sample_train.csv")
train.head()

# # 읽어온 데이터를 code1, code2, similar로 분리
train_code1 = [train["code1"][idx] for idx in range(len(train))]
train_code2 = [train["code2"][idx] for idx in range(len(train))]
train_similar = [train["similar"][idx] for idx in range(len(train))]

In [2]:
# 읽어온 code를 학습용과 검증용으로 분리(8:2)
# 학습용 code에 대한 단어별로 토큰화 후 딕셔너리 생성
# 딕셔너리는 학습용 code의 단어에 대한 index용
# 단어에 대한 index를 RNN 학습을 위한 텐서를 위해 사용
import json

# 사전을 파일에 저장 및 로드
def save_dict(saveDict, fname):
#     filename = 'dict.txt'
    filename = fname
    with open(filename, 'w') as f:
        f.write(json.dumps(saveDict))

def load_dict(filename):
    with open(filename) as f:
        loadDict = json.loads(f.read())
    return loadDict


#code에 대한 토큰화 작업: 학습용 tcode, 검증용 vcode
tcode1 = []
tcode2 = []
tsimilar = []

vcode1 = []
vcode2 = []
vsimilar = []

codedict = {"unknown": 0, "Python padding": 1} # 사전 정의후 새로운 단어에 대해서는 모두 unknown으로 처리

# 읽어온 code에 대해 8:2로 분리후
# 학습용 code에 대해서는 딕셔너리를 생성
# 검증용 code에 대해서는 내용을 알 수 없기에 딕셔너리에 반영 안 함
for idx in range(len(train)):
    if idx%5 == 0:
        vcode1.append(train_code1[idx].split())
        vcode2.append(train_code2[idx].split())
        vsimilar.append(train_similar[idx])
    else:
        tcode1.append(train_code1[idx].split())
    #     dict1 = {key: i for i, key in enumerate(train["code1"][idx].split())}
        for i, key in enumerate(train_code1[idx].split()):
            if key not in codedict:
                codedict[key] = len(codedict)
        tcode2.append(train_code2[idx].split())
    #     dict1 = {key: i for i, key in enumerate(train["code1"][idx].split())}
        for i, key in enumerate(train_code1[idx].split()):
            if key not in codedict:
                codedict[key] = len(codedict)
        tsimilar.append(train_similar[idx])

# code를 분석한 딕셔너리에 대해 검증용 code를 위해 "dict.txt"로 저장
save_dict(codedict, "dict.txt")

In [ ]:
# 저장된 사전파일을 사용하여 코드에 대한 index 작업
# 학습용 코드 index는 tidxcode, 검증용 코드 index는 vidxcode
# 저장된 사전파일에서 로드
idxdict = load_dict("dict.txt")

# RNN 학습에 사용할 Lookup Table에 대한 딕셔너리 단어갯수
n_vocabs = len(idxdict)

# 로드된 딕셔너리를 통해 토큰화된 학습용 code의 index 작업
tidxcode1 = []
tidxcode2 = []

for idx in range(len(tcode1)):
    idxcode1 = []
    for icode in tcode1[idx]:
        try:
            idxcode1.append(idxdict[icode])
        except:
            idxcode1.append(0)
    tidxcode1.append(idxcode1)
    
    idxcode2 = []
    for icode in tcode2[idx]:
        try:
            idxcode2.append(idxdict[icode])
        except:
            idxcode2.append(0)
    tidxcode2.append(idxcode2)

# 로드된 사전을 통해 토큰화된 검증용 code의 index 작업
vidxcode1 = []
vidxcode2 = []

for idx in range(len(vcode1)):
    idxcode1 = []
    for icode in vcode1[idx]:
        try:
            idxcode1.append(idxdict[icode])
        except:
            idxcode1.append(0)
    vidxcode1.append(idxcode1)
    
    idxcode2 = []
    for icode in vcode2[idx]:
        try:
            idxcode2.append(idxdict[icode])
        except:
            idxcode2.append(0)
    vidxcode2.append(idxcode2)

In [3]:
import torch
from torch.utils.data import Dataset

# 1. 데이터셋 클래스 정의

class NSMC_Dataset(Dataset):
  def __init__(self, code1, code2, labels):
    self.code1 = code1
    self.code2 = code2
    self.labels = labels
    self.max_seq_len = 200

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    # idx 번째 데이터 전처리
    # code1, code2 ,label
    labels = self.labels[idx]

    code1 = self.code1[idx]
    code2 = self.code2[idx]

    # 정해진 길이로 맞추기
    # max_seq_len보다 길면 코드의 뒷 부분 제거
    # max_seq_len보다 짧으면 앞에 padding 추가 (Padding의 id는 토크나이저마다 다르지만, 여기서 사용한 토크나이저는 1을 사용)
    if len(code1) > self.max_seq_len:
      code1 = code1[:self.max_seq_len]
    elif len(code1) < self.max_seq_len:
      code1 = [1] * (self.max_seq_len-len(code1)) + code1

    if len(code2) > self.max_seq_len:
      code2 = code2[:self.max_seq_len]
    elif len(code2) < self.max_seq_len:
      code2 = [1] * (self.max_seq_len-len(code2)) + code2


    return torch.LongTensor(code1), torch.LongTensor(code2), torch.FloatTensor([labels])

In [4]:
# 데이터 클래스를 사용 data set을 만들어 학습에 사용할 data loader 생성
from torch.utils.data import DataLoader

# 학습용 데이터 로더
train_dataset = NSMC_Dataset(tidxcode1, tidxcode2, tsimilar)
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle=True)

# 검증용 데이터 로더
val_dataset = NSMC_Dataset(vidxcode1, vidxcode2, vsimilar)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle=False)

In [5]:
# 실행할 device 정의
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [6]:
# 2.RNN 모델 정의

import torch
import torch.nn as nn
import torch.nn.functional as F

class RNN_Text(nn.Module): 
  def __init__(self, n_vocabs):
    super(RNN_Text, self).__init__()

    # LookUp Table 정의
    # 각 id에 해당하는 token들을 100차원 embedding으로 변환
    # 모델과 같이 학습
    self.embs = nn.Embedding(n_vocabs, 100)

    self.hidden_dim = 32
    self.n_layers = 1

    # Pytorch에서 제공하는 RNN 모델
    self.rnn = nn.RNN(100, self.hidden_dim, self.n_layers, batch_first=True)
    
    # RNN의 output을 100차원 embedding으로 변환
    self.fc = nn.Linear(32, 100)

  def forward(self, x):
    batch_size = x.size(0)

    x = self.embs(x)

    hidden = self.init_hidden(batch_size)

    # out: Batch Size X Max Seq Length X Hidden dim
    out, hidden = self.rnn(x, hidden) 
    # out: Batch Size X Hidden dim (시퀀스의 마지막 embedding만 사용(문장 전체의 의미를 가지고 있기 때문))
    out = self.fc(out[:,-1])
    
    return out, hidden
  
  # hidden_0을 만드는 함수
  def init_hidden(self, batch_size):
    hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
    return hidden

In [7]:
# RNN 클래스 인스턴스 생성과 로스기준 및 오티마이저 인스턴스 생성
import torch.optim as optim

rnn_model = RNN_Text(n_vocabs*4)
rnn_model = rnn_model.to(device)

criterion = nn.BCELoss()
rnn_optimizer = optim.Adam(rnn_model.parameters())

In [10]:
# 3. 모델 학습
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

cos = nn.CosineSimilarity(dim=1, eps=1e-4)
    
for epoch_i in range(20):  # 에포크 반복
  avg_loss = 0.0
  # for batch in train_loader:
  rnn_model.train()

  for batch in tqdm(train_loader): # DataLoader의 code1에 대한 배치

    inputs1, inputs2, labels = batch[0], batch[1], batch[2]
    inputs1 = inputs1.to(device)
    inputs2 = inputs2.to(device)
    labels = labels.to(device)
    
    rnn_optimizer.zero_grad() # 기울기 초기화
    
    outputs1, _ = rnn_model(inputs1) # 모델 forward
    outputs2, _ = rnn_model(inputs2) # 모델 forward
    
    
# # cosine 유사도 계산에서 극단적인 값이 나올 수 있으므로 정규화
    norm_output1 = torch.linalg.norm(outputs1, dim=1)
    norm_output2 = torch.linalg.norm(outputs2, dim=1)

    outputs1 = outputs1 / norm_output1.unsqueeze(1)
    outputs2 = outputs2 / norm_output2.unsqueeze(1)

    # cosine 유사도 구하기
    out = cos(outputs1, outputs2)

    out = (out + 1) / 2.1 # 범위를 -1 ~ 1 => 0 ~ 1 로 변경

    loss = criterion(out, labels.flatten()) # loss 계산

    avg_loss += loss.item() # loss 값만 누적

    loss.backward() # loss를 통해서 기울기 계산

    rnn_optimizer.step() # 모델 업데이트
    

  print(f'{epoch_i+1}: {avg_loss/len(train_loader)}')

torch.save(rnn_model.state_dict(), 'DictRNN')

100%|█████████████████████████████████████████| 450/450 [01:08<00:00,  6.56it/s]


1: 0.055553656390143764


100%|█████████████████████████████████████████| 450/450 [01:10<00:00,  6.38it/s]


2: 0.05166832056310442


100%|█████████████████████████████████████████| 450/450 [01:11<00:00,  6.32it/s]


...

18: 0.04489193956471152


100%|█████████████████████████████████████████| 450/450 [01:08<00:00,  6.57it/s]


19: 0.04514520647625128


100%|█████████████████████████████████████████| 450/450 [01:09<00:00,  6.51it/s]

20: 0.044676762552311025


In [8]:
# 3. 모델 학습 및 평가
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# cosine similarity 계산을 위한 객체 생성
cos = nn.CosineSimilarity(dim=1, eps=1e-4)

# 모델 학습 및 평가
for epoch_i in range(10):
  print(f'Train epoch {epoch_i+1}: ')
  avg_loss = 0.0

# 모델 학습
  rnn_model.train()

  # 학습 데이터로 학습
  for batch in tqdm(train_loader):
    code1_ids, code2_ids, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)

    rnn_optimizer.zero_grad() # 기울기 초기화

    outputs1, _ = rnn_model(code1_ids) # code1 임베딩
    outputs2, _ = rnn_model(code2_ids) # code2 임베딩
    
    # cosine 유사도 계산에서 극단적인 값이 나올 수 있으므로 정규화
    norm_output1 = torch.linalg.norm(outputs1, dim=1)
    norm_output2 = torch.linalg.norm(outputs2, dim=1)

    outputs1 = outputs1 / norm_output1.unsqueeze(1)
    outputs2 = outputs2 / norm_output2.unsqueeze(1)

    # cosine 유사도 구하기
    out = cos(outputs1, outputs2)

    # 유사도를 0 ~ 1 범위로 정규화
    out = (out + 1) / 2.1 # 범위를 -1 ~ 1 => 0 ~ 1 로 변경

    loss = criterion(out, labels.flatten()) # loss 계산
    
    avg_loss += loss.item() # loss 값만 누적

    loss.backward() # loss를 통해서 기울기 계산

    rnn_optimizer.step() # 모델 업데이트

#   print(f'loss {avg_loss/len(train_loader)}')

# 모델 검증
  rnn_model.eval()

  correct = 0

  # 검증 데이터로 평가
  for batch in tqdm(val_loader):
    code1_ids, code2_ids, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device)

    with torch.no_grad():
      outputs1, _ = rnn_model(code1_ids) # code1 임베딩
      outputs2, _ = rnn_model(code2_ids) # code2 임베딩
      
      pred = cos(outputs1, outputs2)
      pred = (pred + 1) / 2
      
      pred[pred > 0.5] = 1.0
      pred[pred <= 0.5] = 0.0

      correct += torch.sum(pred == labels.flatten())

#   print(f'Val accuracy {correct/len(val_dataset)}')
  print(f'loss {avg_loss/len(train_loader)}, Val accuracy {correct/len(val_dataset)}')
  print()

torch.save(rnn_model.state_dict(), 'DictRNN')
print('Done')

Train epoch 1: 


100%|█████████████████████████████████████████| 113/113 [00:01<00:00, 68.55it/s]


loss 0.6662318835655848, Val accuracy 0.684195876121521

Train epoch 2: 


100%|█████████████████████████████████████████| 113/113 [00:01<00:00, 71.12it/s]


loss 0.38291754136482875, Val accuracy 0.9488035440444946

Train epoch 3: 


...

loss 0.06698160926500957, Val accuracy 0.9849749803543091

Train epoch 9: 


100%|█████████████████████████████████████████| 113/113 [00:01<00:00, 68.37it/s]


loss 0.06544785767379735, Val accuracy 0.987200915813446

Train epoch 10: 


100%|█████████████████████████████████████████| 113/113 [00:01<00:00, 92.73it/s]


loss 0.06700875616735882, Val accuracy 0.9849749803543091

Done


In [2]:
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.data import Dataset
import torch
import numpy as np
import pandas as pd
import json
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn as nn
import torch.nn.functional as F

print('Reading data...')
# 학습된 모델로 code 유사성 판단
submission = pd.read_csv('open/test.csv')
submission.head()

# 읽어온 데이터를 code1, code2, similar로 분리
test_code1 = [submission["code1"][idx] for idx in range(len(submission))]
test_code2 = [submission["code2"][idx] for idx in range(len(submission))]
test_pairs = [submission["pair_id"][idx] for idx in range(len(submission))]

fcode1 = []
fcode2 = []
for idx in range(len(submission)):
    fcode1.append(test_code1[idx].split())
    fcode2.append(test_code2[idx].split())

print('Data tokenizing...')
# code에 대한 index
def load_dict(filename):
    with open(filename) as f:
        loadDict = json.loads(f.read())
    return loadDict

# 저장된 사전파일에서 로드
idxdict = load_dict("dict.txt")

# 로드된 사전을 통해 토큰화된 유사성 판단용 code의 index 작업
fidxcode1 = []
fidxcode2 = []

for idx in range(len(test_code1)):
    idxcode1 = []
    for icode in fcode1[idx]:
        try:
            idxcode1.append(idxdict[icode])
        except:
            idxcode1.append(0)
    fidxcode1.append(idxcode1)
    
    idxcode2 = []
    for icode in fcode2[idx]:
        try:
            idxcode2.append(idxdict[icode])
        except:
            idxcode2.append(0)
    fidxcode2.append(idxcode2)

print('Define class of Dataset & RNN_Text')
    
class FIN_Dataset(Dataset):
  def __init__(self, code1, code2):
    self.code1 = code1
    self.code2 = code2
    self.max_seq_len = 200

  def __len__(self):
    return len(self.code1)

  def __getitem__(self, idx):
    code1 = self.code1[idx]
    code2 = self.code2[idx]

    if len(code1) > self.max_seq_len:
      code1 = code1[:self.max_seq_len]
    elif len(code1) < self.max_seq_len:
      code1 = [1] * (self.max_seq_len-len(code1)) + code1

    if len(code2) > self.max_seq_len:
      code2 = code2[:self.max_seq_len]
    elif len(code2) < self.max_seq_len:
      code2 = [1] * (self.max_seq_len-len(code2)) + code2

    return torch.LongTensor(code1), torch.LongTensor(code2)

# 테스트 데이터 로더
final_dataset = FIN_Dataset(fidxcode1, fidxcode2)
final_loader = DataLoader(final_dataset, batch_size = 32, shuffle=False)

class RNN_Text(nn.Module): 
  def __init__(self, n_vocabs):
    super(RNN_Text, self).__init__()

    # LookUp Table 정의
    # 각 id에 해당하는 token들을 100차원 embedding으로 변환
    # 모델과 같이 학습
    self.embs = nn.Embedding(n_vocabs, 100)

    self.hidden_dim = 32
    self.n_layers = 1

    # Pytorch에서 제공하는 RNN 모델
    self.rnn = nn.RNN(100, self.hidden_dim, self.n_layers, batch_first=True)
    
    # RNN의 output을 100차원 embedding으로 변환
    self.fc = nn.Linear(32, 100)

  def forward(self, x):
    batch_size = x.size(0)

    x = self.embs(x)

    hidden = self.init_hidden(batch_size)

    # out: Batch Size X Max Seq Length X Hidden dim
    out, hidden = self.rnn(x, hidden) 
    # out: Batch Size X Hidden dim (시퀀스의 마지막 embedding만 사용(문장 전체의 의미를 가지고 있기 때문))
    out = self.fc(out[:,-1])
    
    return out, hidden
  
  # hidden_0을 만드는 함수
  def init_hidden(self, batch_size):
    hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
    return hidden

print('Starting validation data...')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dict_model = RNN_Text(len(idxdict)*4)
dict_model.load_state_dict(torch.load('DictRNN'))
dict_model.eval()

# rnn_model.eval()

# cosine similarity 계산을 위한 객체 생성
cos = nn.CosineSimilarity(dim=1, eps=1e-4)
fc = nn.Linear(32, 1)
preds = []

# 테스트 데이터로 평가
for batch in tqdm(final_loader):
    code1_ids, code2_ids = batch[0].to(device), batch[1].to(device)

    with torch.no_grad():
      outputs1, _ = dict_model(code1_ids) # code1 임베딩
      outputs2, _ = dict_model(code2_ids) # code2 임베딩

      pred = cos(outputs1, outputs2)
      pred = (pred + 1) / 2

      pred[pred > 0.5] = 1.0
      pred[pred <= 0.5] = 0.0
      pred = pred.numpy()

      for icnt in range(len(pred)):
          preds.append(int(pred[icnt]))

submission['pred'] = preds
submission.to_csv('open/testDict.csv', index=False)

print('Done')

Reading data...
Data tokenizing...
Define class of Dataset & RNN_Text
Starting validation data...


100%|██████████████████████████████████████| 5616/5616 [00:54<00:00, 102.47it/s]


Done
